In [ ]:
#Login to the MSP SASE portal
import prisma_sase
from time import sleep

PRISMASASE_CLIENT_ID = "xxxxx"
PRISMASASE_CLIENT_SECRET = "xxxxxx"
 
tsgid = "xxxxx"

sdk = prisma_sase.API(controller="https://api.sase.paloaltonetworks.com", ssl_verify=False)
sdk.set_debug(3)
sdk.interactive.login_secret(client_id=PRISMASASE_CLIENT_ID, client_secret=PRISMASASE_CLIENT_SECRET, tsg_id=tsgid) 


In [ ]:
#Create a multi-tenant incidents notification profile
url = "https://api.sase.paloaltonetworks.com/mt/notifications/v1/profiles"
data = {
    "profileName": "Tunnel-state-notification-profile",
    "tenantList":
    [
        "1xxxxx3",
        "16xxxx"
    ],
    "description": "Profile for notifications on tunnel state down,flap",
    "excludeTenantList": [],
    
    "notifChannels":
    [
        {
            "name": "NOC-US-admin",
            "type": "WEBHOOK",
            "webhookChannelDetails":
            {
                "urls":
                [
                    "https://webhook.site/ce5d06xxxxx9",
                    
                ],
                "authType": "NO_AUTH",
                

            }
        },
        {
            "emailChannelDetails":
            {
                "emails":
                [
                    {
                        "emailId": "xxxxx@gmail.com",
                        "name": "NOC-US-admin"
                    }
                ]
            },
            "name": "NOC-admin-US",
            "type": "EMAIL"
        }
    ],
    "notifTypeDetails": [
            
                {
                "notifCategoryList": [
                    {
                        "bestPractice": False,
                        "name": "RN",
                        "subCategoryList": [
                            {
                                "bestPractice": False,
                                "name": "INC_RN_ECMP_TUNNEL_DOWN"
                            },
                            {
                                "bestPractice": False,
                                "name": "INC_RN_SECONDARY_WAN_TUNNEL_RTT_EXCEEDED_BASELINE"
                            },
                            {
                                "bestPractice": False,
                                "name": "INC_RN_ECMP_BGP_DOWN"
                            }
                        ]
                    }
                ],
                "type": "INCIDENTS"
            }
        ],
    "opState": "ENABLED",
}


resp = sdk.rest_call(method="POST", url=url, data=data)
prisma_sase.jd_detailed(resp)



In [ ]:
#Creating a Multi tenant Upgrades/Announcements notification profile
url = "https://api.sase.paloaltonetworks.com/mt/notifications/v1/profiles"
data = {
    "profileName": "upgrades-announcements-profile",
    "tenantList":
    [
        "1xxxxx",
        "1xxxx"
    ],
    "description": "Profile for Upgrades/Announcements",
    "excludeTenantList": [],
    
    "notifChannels":
    [
        {
            "name": "NOC-US-admin",
            "type": "WEBHOOK",
            "webhookChannelDetails":
            {
                "urls":
                [
                    "https://webhook.site/ce5d0687-0175-42b0-9d3a-c4423bb97799",
                    
                ],
                "authType": "NO_AUTH",
                

            }
        },
        {
            "emailChannelDetails":
            {
                "emails":
                [
                    {
                        "emailId": "xxxxx@gmail.com",
                        "name": "NOC-US-admin"
                    }
                ]
            },
            "name": "NOC-admin-US",
            "type": "EMAIL"
        }
    ],
    "notifTypeDetails": [
                {
                    "notifCategoryList": [
                        {
                            "bestPractice": False,
                            "name": "Dataplane",
                            "subCategoryList": [
                                {
                                    "bestPractice": True,
                                    "name": "In Progress"
                                },
                                {
                                    "bestPractice": True,
                                    "name": "Error"
                                }
                            ]
                        }
                    ],
                    "type": "UPGRADES"
                },
                
        ],
    "opState": "ENABLED",
}


resp = sdk.rest_call(method="POST", url=url, data=data)
prisma_sase.jd_detailed(resp)

In [ ]:
#List all the notifications profiles (Using this to fetch a specific profile to edit in the next API call)
base_url = "https://api.sase.paloaltonetworks.com/mt/notifications/v1/profiles"
resp = sdk.rest_call(method="GET", url=url)
prisma_sase.jd_detailed(resp)

In [ ]:
#Update/edit the Upgrades/Announcements notification profile to add announcements 
#IMP note: PUT needs old payload too, else use PATCH
id = "7a972b3f-73exxxxxxx"
url = "https://api.sase.paloaltonetworks.com/mt/notifications/v1/profiles/"+id
data = {
    "profileName": "upgrades-announcements-profile",
    "tenantList":
    [
        "1xxxx",
        "1xxxxx"
    ],
    "description": "Profile for Upgrades/Announcements",
    "excludeTenantList": [],
    "id": id,
    "notifChannels":
    [
      
        {
            "emailChannelDetails":
            {
                "emails":
                [
                    {
                        "emailId": "xxxxx@gmail.com",
                        "name": "NOC-US-admin"
                    },
                    

                    {
                        "emailId": "xxxx@gmail.com",
                        "name": "NOC-US-admin"
                    }
                ]
            },
            "name": "NOC-admin-US",
            "type": "EMAIL"
        }
    ],
     "notifTypeDetails": [
        {
        "type": "ANNOUNCEMENTS",
      
        },
        {
                    "notifCategoryList": [
                        {
                            "bestPractice": False,
                            "name": "Dataplane",
                            "subCategoryList": [
                                {
                                    "bestPractice": True,
                                    "name": "In Progress"
                                },
                                {
                                    "bestPractice": True,
                                    "name": "Error"
                                }
                            ]
                        }
                    ],
                    "type": "UPGRADES"
                },
    ],
    "opState": "ENABLED",
}


resp = sdk.rest_call(method="PUT", url=url, data=data)
prisma_sase.jd_detailed(resp)


In [ ]:
#Fetch all upgrade/announcement notifications in read/unread state in ascending order of impacted tenants


url = "https://api.sase.paloaltonetworks.com/mt/notifications/v1/list"
data = {
  "filters": [
    {
      "field": "readState",
      "values": ["READ","UNREAD"]
    },
    {
      "field": "notifType",
      "values": ["UPGRADES", "ANNOUNCEMENTS"]
    }
  ],
  "sortByList": [{
    "field": "impactedTenantCount",
    "sortBy": "ASC"
  }],
  "page":{
            "num": 0,
            "size": 10
          }
}


resp = sdk.rest_call(method="POST", url=url, data=data)
prisma_sase.jd_detailed(resp)




In [ ]:
#Fetch all incident notifications in read/unread state in ascending order of impacted tenant

url = "https://api.sase.paloaltonetworks.com/mt/monitor/v1/agg/incidents/list"
data={
    "properties":
          [
            {"property":"incident_id"},
            {"property":"title"},
            {"property":"sub_tenant_id"},
            {"property":"severity"},
            {"property":"category"},
            {"property":"sub_category"},
            {"property":"status"},
            {"property":"created_time"},
            {"property":"updated_time"},
            {"property":"clear_reason"},
            {"property":"code"}
            ],
            "filter":
            {"operator":"AND",
             "rules":
               [
                 {"property":"updated_time",
                 "operator":"last_n_days",
                            "values":[7]
                 },
                {"property":"domain",
                 "operator":"in",
                 "values":["External","external"]
                },
                {"property":"severity",
                "operator":"in",
                "values":["Critical","Warning"]
                
                }
                ]
              }
            }        

resp = sdk.rest_call(method="POST", url=url, data=data)
prisma_sase.jd_detailed(resp)



